## Spotify data ETL project

### Imports

In [227]:
import pandas as pd
import sqlalchemy
from sqlalchemy.orm import sessionmaker
import requests
import json
import datetime
from datetime import datetime, timedelta
import sqlite3

In [228]:
# constants
DB_LOCATION = 'sqlite:///spoty_songs.sqlite'
USER_ID = 'Hristiyan'
SPOTY_TOKEN = 'BQCalp-OX0Y3WXgEjDgb0a-HRPTewHhS0K3WeU4kUOdTvD4PpAmW1Ou64w0ASuunavi0mgJzKXTMI3FZwcf0eyeKtuwFgvDXQ-V6c4xttkKD4P2XjjurzoMvV2VcSZKAq3htNnFLVyczH-yze3V36P2oZ-kDM9sX4VK9Eyk2Y47msiPhEFEbfn3ctQ8OQRmFxn8_Jw'

In [229]:
# create data validation function
def data_validation(df: pd.DataFrame) -> bool:
    if df.empty:
        print('No songs downloaded. Finishing execution')
        return False

    # Primary Key Check
    if pd.Series(df['played_at']).is_unique:
        pass
    else:
        raise Exception('Primary Key Check is violated')

    # Check for nulls
    if df.isnull().values.any():
        raise Exception('Null values found')

    # in case we want to check that the timestamp is from any given day
    """
    # Check that all timestamps are created yesterday 
    yesterday = datetime.now() - timedelta(days=1)
    yesterday = yesterday.replace(hour=0, minute=0, second=0, microsecond=0)

    timestamps = df['timestamp'].tolist()
    for timestamp in timestamps:
        if datetime.strptime(timestamp, '%Y-%m-%d') != yesterday:
            raise Exception('At least one of the returned songs does not come from yesterday')
    """

    return True

In [230]:
# check if current script is being run as main program
if __name__ == "__main__":
    # create the headers for the request
    headers = {
        "Accept" : "application/json",
        "Content-Type" : "application/json",
        "Authorization" : "Bearer {token}".format(token=SPOTY_TOKEN)
    }

In [231]:
# define yesterday unix timestamp step by step
today = datetime.now()
yesterday = datetime.now() - timedelta(days=60)
yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000

In [232]:
# request the data from the API
r = requests.get("https://api.spotify.com/v1/me/player/recently-played?after={time}".format(time=yesterday_unix_timestamp), headers=headers)

In [233]:
# putting data into json format
data = r.json()
# print(data['items'][0])

In [234]:
# create the lists needed for the dataframe
song_names = []
artist_names = []
played_at_list = []
timestamps = []

In [235]:
# loop through the data and append to the lists
for song in data["items"]:
    song_names.append(song["track"]["name"])
    artist_names.append(song["track"]["album"]["artists"][0]["name"])
    played_at_list.append(song["played_at"])
    timestamps.append(song["played_at"][0:10])

In [236]:
# create the dictionary of the data
song_dict = {
    "song_name" : song_names,
    "artist_name" : artist_names,
    "played_at" : played_at_list,
    "timestamp" : timestamps
}

In [237]:
# create the dataframe
songs_df = pd.DataFrame(song_dict, columns = ["song_name", "artist_name", "played_at", "timestamp"])

# to check df
songs_df.head()

# to check for specific artist
#songs_df[songs_df['artist_name'] == '{Artist Name}'].head()

,song_name,artist_name,played_at,timestamp
0,Parti,Mikri Maus,2023-03-04T18:48:48.232Z,2023-03-04
1,I ti i ti,Juice,2023-03-04T18:45:18.008Z,2023-03-04
2,I ti i ti,Juice,2023-03-04T18:44:07.222Z,2023-03-04
3,Jahtung,Mikri Maus,2023-03-04T18:44:07.021Z,2023-03-04
4,Nebo,Connect,2023-03-04T18:42:27.956Z,2023-03-04


In [238]:
# validate the data using the data validation function
if data_validation(songs_df):
    print('Data valid, proceed to Load stage')

Data valid, proceed to Load stage


In [239]:
# create/update the database
engine = sqlalchemy.create_engine(DB_LOCATION)
conn = sqlite3.connect('spoty_songs.sqlite')
cursor = conn.cursor()

sql_query = """
CREATE TABLE IF NOT EXISTS spoty_songs (
    song_name VARCHAR(200),
    artist_name VARCHAR(200),
    played_at VARCHAR(200),
    timestamp VARCHAR(200),
    CONSTRAINT primary_key_constraint PRIMARY KEY (played_at)
)
"""

cursor.execute(sql_query)
print("Created/updated database successfully")

Created/updated database successfully


In [240]:
# load data from the df into the database
try:
    songs_df.to_sql("spoty_songs", engine, index=False, if_exists='append')
    print("Data appended successfully")
except sqlalchemy.exc.IntegrityError:
    print("Data already exists in the database")
except Exception as e:
    print(f"An error occurred: {e}")

conn.close()
print("Closed database successfully")

Data appended successfully
Closed database successfully
